In [10]:
!python ../search_with_machine_learning_course/week2/createContentTrainingData.py --input ../datasets/product_data/products/ --output ../datasets/fasttext/labeled_products.txt


Writing results to ../datasets/fasttext/labeled_products.txt
100%|█████████████████████████████████████████| 256/256 [00:31<00:00,  8.24it/s]


In [13]:
!shuf ../datasets/fasttext/labeled_products.txt > ../datasets/fasttext/shuffled_labeled_products.txt


In [21]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))


In [14]:
!head -n 10000 ../datasets/fasttext/shuffled_labeled_products.txt > ../datasets/fasttext/training_data.txt
!tail -n 10000 ../datasets/fasttext/shuffled_labeled_products.txt > ../datasets/fasttext/test_data.txt

In [36]:
%%time
import fasttext
classifier1 = fasttext.train_supervised(input="../datasets/fasttext/training_data.txt")
print_results(*classifier1.test("../datasets/fasttext/test_data.txt"))


Read 0M words
Number of words:  11037
Number of labels: 1327
Progress:  96.2% words/sec/thread:   27252 lr:  0.003785 avg.loss: 13.435069 ETA:   0h 0m 0s

N	9571
P@1	0.105
R@1	0.105
CPU times: user 20.8 s, sys: 58 ms, total: 20.9 s
Wall time: 3.2 s


Progress: 100.0% words/sec/thread:   27047 lr:  0.000000 avg.loss: 13.328848 ETA:   0h 0m 0s


## Parameter Tunning

In [35]:
%%time
classifier2 = fasttext.train_supervised(input="../datasets/fasttext/training_data.txt",lr=1,epoch=25)
print_results(*classifier2.test("../datasets/fasttext/test_data.txt"))


Read 0M words
Number of words:  11037
Number of labels: 1327
Progress: 100.0% words/sec/thread:   26919 lr:  0.000000 avg.loss:  1.125730 ETA:   0h 0m 0s


N	9571
P@1	0.637
R@1	0.637
CPU times: user 1min 42s, sys: 170 ms, total: 1min 42s
Wall time: 12.5 s


In [37]:
%%time
classifier3 = fasttext.train_supervised(input="../datasets/fasttext/training_data.txt",lr=1,epoch=25,wordNgrams=2)
print_results(*classifier3.test("../datasets/fasttext/test_data.txt"))


Read 0M words
Number of words:  11037
Number of labels: 1327
Progress: 100.0% words/sec/thread:   27162 lr:  0.000000 avg.loss:  1.442676 ETA:   0h 0m 0s


N	9571
P@1	0.623
R@1	0.623
CPU times: user 1min 43s, sys: 315 ms, total: 1min 43s
Wall time: 12.8 s


## Content Normalization
 1. Remove all non-alphanumeric characters other than underscore (which we need for the labels!).
 2. Convert all letters to lowercase.
 3. Trim excess space characters so that tokens are separated by a single space.

In [41]:
!cat ../datasets/fasttext/training_data.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > ../datasets/fasttext/normalized_training_lite.txt
!cat ../datasets/fasttext/test_data.txt |sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]_]/ /g" | tr -s ' ' > ../datasets/fasttext/normalized_test_lite.txt



In [44]:
%%time
classifier4 = fasttext.train_supervised(input="../datasets/fasttext/normalized_training_lite.txt",lr=1,epoch=25,wordNgrams=2)
print_results(*classifier4.test("../datasets/fasttext/normalized_test_lite.txt"))

Read 0M words
Number of words:  8609
Number of labels: 1327
Progress: 100.0% words/sec/thread:   26178 lr: -0.000081 avg.loss:  1.356068 ETA:   0h 0m 0s

N	9571
P@1	0.617
R@1	0.617
CPU times: user 1min 40s, sys: 397 ms, total: 1min 41s
Wall time: 12.6 s


Progress: 100.0% words/sec/thread:   26178 lr:  0.000000 avg.loss:  1.356068 ETA:   0h 0m 0s


## Steming 

In [49]:
!python ../search_with_machine_learning_course/week2/createContentTrainingData.py --input ../datasets/product_data/products/ --output ../datasets/fasttext/stemmed_labeled_products.txt
!shuf ../datasets/fasttext/stemmed_labeled_products.txt > ../datasets/fasttext/shuffled_stemmed_labeled_products.txt



Writing results to ../datasets/fasttext/stemmed_labeled_products.txt
100%|█████████████████████████████████████████| 256/256 [00:33<00:00,  7.58it/s]


In [50]:
!head -n 10000 ../datasets/fasttext/shuffled_stemmed_labeled_products.txt > ../datasets/fasttext/stemmed_training_data.txt
!tail -n 10000 ../datasets/fasttext/shuffled_stemmed_labeled_products.txt > ../datasets/fasttext/stemmed_test_data.txt


In [52]:
%%time
import fasttext
classifier5 = fasttext.train_supervised(input="../datasets/fasttext/stemmed_training_data.txt",lr=1,epoch=25)
print_results(*classifier5.test("../datasets/fasttext/stemmed_test_data.txt"))


Read 0M words
Number of words:  11080
Number of labels: 1366
Progress: 100.0% words/sec/thread:   25835 lr:  0.000000 avg.loss:  1.189853 ETA:   0h 0m 0s


N	9698
P@1	0.623
R@1	0.623
CPU times: user 1min 47s, sys: 187 ms, total: 1min 47s
Wall time: 13.1 s


## Pruning 

In [65]:
filepath = '../datasets/fasttext/labeled_products.txt'
entries = []
with open(filepath) as fp:
   line = fp.readline()
   while line:
       vals = line.split(" ", 1)
       # print(f"{len(vals)} -> {vals[0]}-{vals[1]}")
       line = fp.readline()
       entries.append((vals[0], vals[1]))

print(len(entries))
print(entries[0])

115503
('__label__pcmcat209400050001', 'LG - Nitro HD 4G Mobile Phone - Black (AT&T)\n')


In [66]:
import pandas as pd
df = pd.DataFrame(entries, columns =['Category', 'Name'])

In [70]:
cat_count = df.groupby(['Category']).Name.count()


In [71]:
pruned_cats_500 = cat_count[cat_count > 500].index.tolist()
print(len(pruned_cats_500))
pruned_cats_100 = cat_count[cat_count > 100].index.tolist()
print(len(pruned_cats_100))

32
262


In [73]:
input_filepath = '../datasets/fasttext/labeled_products.txt'
output_filepath = '../datasets/fasttext/pruned_labeled_products_500.txt'
ip_file = open(input_filepath, 'r')
op_file = open(output_filepath, 'w')

line = ip_file.readline()
while line:
   vals = line.split(" ", 1)
   line = ip_file.readline()
   if vals[0] in pruned_cats_500:
       op_file.write(line)

ip_file.close()
op_file.close()

In [75]:
!shuf ../datasets/fasttext/pruned_labeled_products_500.txt > ../datasets/fasttext/shuffled_pruned_labeled_products_500.txt


In [76]:
!head -n 10000 ../datasets/fasttext/shuffled_pruned_labeled_products_500.txt > ../datasets/fasttext/prune_500_training_data.txt
!tail -n 10000 ../datasets/fasttext/shuffled_pruned_labeled_products_500.txt > ../datasets/fasttext/prune_500_test_data.txt


In [77]:
%%time
import fasttext
classifier6 = fasttext.train_supervised(input="../datasets/fasttext/prune_500_training_data.txt",lr=1,epoch=25)
print_results(*classifier6.test("../datasets/fasttext/prune_500_test_data.txt"))


Read 0M words
Number of words:  9226
Number of labels: 992
Progress: 100.0% words/sec/thread:   35629 lr:  0.000000 avg.loss:  0.750655 ETA:   0h 0m 0s


N	9656
P@1	0.756
R@1	0.756
CPU times: user 1min 19s, sys: 233 ms, total: 1min 19s
Wall time: 10 s


## Synonymns

In [79]:
!cut -d' ' -f2- ../datasets/fasttext/shuffled_labeled_products.txt > ../datasets/fasttext/titles.txt


In [82]:
%%time
model = fasttext.train_unsupervised('../datasets/fasttext/titles.txt', model='skipgram')
#model.save_model("title_model.bin")

Read 1M words
Number of words:  10873
Number of labels: 0
Progress:  99.5% words/sec/thread:  203514 lr:  0.000257 avg.loss:  1.363375 ETA:   0h 0m 0s

CPU times: user 28.2 s, sys: 321 ms, total: 28.5 s
Wall time: 3.84 s


Progress: 100.0% words/sec/thread:  198348 lr:  0.000000 avg.loss:  1.361880 ETA:   0h 0m 0s


In [83]:
model.get_nearest_neighbors('iphone')


[(0.8632429242134094, 'Saxophone'),
 (0.8432313203811646, 'Speakerphone'),
 (0.8210281729698181, 'Microphone'),
 (0.8071489930152893, 'Telephone'),
 (0.8069990277290344, 'Earphones'),
 (0.7941811084747314, 'Ozone'),
 (0.7836856245994568, 'Headphone'),
 (0.7751611471176147, 'GoPhone'),
 (0.7704157829284668, 'Phone'),
 (0.7541067004203796, 'Telephones')]

In [84]:
model.get_nearest_neighbors('iPhone')


[(0.955690324306488, 'iPhone®,'),
 (0.940699577331543, 'iPhone®'),
 (0.836539089679718, 'iPod®/iPhone'),
 (0.8291558027267456, '4S'),
 (0.820639967918396, 'iPod®/iPhone®'),
 (0.8167041540145874, '3GS'),
 (0.8142791986465454, '3G,'),
 (0.8122727870941162, '3GS,'),
 (0.808302104473114, '4/4S'),
 (0.804758608341217, '3G/3GS')]

In [85]:
!cat ../datasets/fasttext/titles.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" | sed "s/[^[:alnum:]]/ /g" | tr -s ' ' > ../datasets/fasttext/normalized_titles.txt


In [87]:
model_norm = fasttext.train_unsupervised('../datasets/fasttext/normalized_titles.txt', model='skipgram')


Read 1M words
Number of words:  8903
Number of labels: 0
Progress: 100.0% words/sec/thread:  206268 lr:  0.000000 avg.loss:  1.474714 ETA:   0h 0m 0s


In [88]:
model_norm.get_nearest_neighbors('iphone')


[(0.8629015684127808, '4s'),
 (0.771943211555481, '3gs'),
 (0.7652565836906433, 'apple'),
 (0.7335160970687866, 'ozone'),
 (0.7167626619338989, 'ifrogz'),
 (0.7095106840133667, 'ipadâ'),
 (0.7040247917175293, 'gophone'),
 (0.7027230858802795, 'amethyst'),
 (0.696928083896637, 'appleâ'),
 (0.6963557600975037, 'ipod')]

In [89]:
def get_neighbours(model, query, threshold=0.1):
    results = model.get_nearest_neighbors(query)
    neighbours = []
    for result in results:
        if result[0] >= threshold:
            neighbours.append(result[1])
    return neighbours

In [90]:
model_epochs25 = fasttext.train_unsupervised('../datasets/fasttext/normalized_titles.txt', model='skipgram',epoch=25)


Read 1M words
Number of words:  8903
Number of labels: 0
Progress: 100.0% words/sec/thread:  210602 lr:  0.000000 avg.loss:  1.074682 ETA:   0h 0m 0s


In [92]:
query_list = ['iphone', 'headphones', 'laptop', 'freezer', 'nintendo', 'whirlpool', 'kodak', 'ps2', 'razr', 'stratocaster', 'holiday', 'plasma', 'leather']
for query in query_list:
    print(query,get_neighbours(model_epochs25,query)[0:5])


iphone ['4s', 'apple', 'ipod', '3gs', 'ipad']
headphones ['earbud', 'headphone', 'ear', 'earphones', 'earpollution']
laptop ['netbook', 'notebook', '176', '177', 'zenbook']
freezer ['freezers', 'refrigerator', 'cu', 'side', 'mug']
nintendo ['ds', 'wii', 'nintendogs', '3ds', 'gamecube']
whirlpool ['biscuit', 'maytag', 'frigidaire', 'ge', 'inglis']
kodak ['easyshare', 'm763', 'c813', 'm893', 'm863']
ps2 ['playstation', 'ps3', 'gamecube', 'xbox', 'guide']
razr ['krzr', 'motorola', 'droid', 'a855', 't720']
stratocaster ['telecaster', 'starcaster', 'fretboard', 'strat', 'fender']
holiday ['holidays', 'nobr', 'vibes', 'despicable', 'dreidel']
plasma ['600hz', 'hdtv', 'viera', '480hz', 'purevision']
leather ['leatherskin', 'recliner', 'hipcase', 'armless', 'maccase']


In [95]:
!cat ../datasets/fasttext/normalized_titles.txt | tr " " "\n" | grep "...." | sort | uniq -c | sort -nr | head -1000 | grep -oE '[^ ]+$' >  ../datasets/fasttext/top_words.txt


sort: Broken pipe


In [108]:
filepath = '../datasets/fasttext/top_words.txt'
op_file = open('../datasets/fasttext/synonyms.csv','w')

with open(filepath) as fp:
   line = fp.readline()
   while line:
       line = line.strip()
       synonyms = get_neighbours(model_epochs25, line, 0.65)
       synonym_entry = f"{line},{','.join(synonyms)}\n"
       op_file.write(synonym_entry)
       line = fp.readline()

op_file.close()

In [110]:
!cp ../datasets/fasttext/synonyms.csv ../search_with_machine_learning_course/docker/config/
    